In [41]:
with open("./test_qa.txt",encoding='utf-8') as f:
    real_estate_sales = f.read()

In [42]:
pip install faiss-cpu==1.7.4

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [43]:
from langchain.text_splitter import CharacterTextSplitter

In [44]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [45]:
docs = text_splitter.create_documents([real_estate_sales])

In [46]:
docs[0]

Document(page_content='[客户问题] 这辆车的油耗是多少？\n[销售回答] 这款车型在城市驾驶条件下平均油耗约为8升/100公里，在高速公路上则降低至6升/100公里。')

In [47]:
len(docs)

50

### 使用 Faiss 作为向量数据库，持久化存储汽车销售 问答对（QA-Pair）

In [48]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [49]:
query = "请问车辆支持无线carplay吗？"

In [50]:
answer_list = db.similarity_search(query)

In [51]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 车辆支持Apple CarPlay和Android Auto吗？
[销售回答] 是的，信息娱乐系统支持这两种手机连接方式，方便您在驾驶时使用手机功能。

[客户问题] 车辆是否支持无线充电？
[销售回答] 是的，中央控制台有一个无线充电板，支持Qi标准的设备都可以使用。

[客户问题] 车辆是否支持远程启动？
[销售回答] 是的，通过智能手机应用或遥控钥匙，您可以远程启动发动机并预设车内温度。

[客户问题] 这车的蓝牙连接最多支持几部设备？
[销售回答] 蓝牙系统通常支持同时连接两部设备，便于切换不同用户的电话和音乐播放。



In [52]:
db.save_local("car_sales_qa")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [53]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [54]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000207B8188BE0>, search_kwargs={'k': 3})

In [55]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 车辆支持Apple CarPlay和Android Auto吗？
[销售回答] 是的，信息娱乐系统支持这两种手机连接方式，方便您在驾驶时使用手机功能。

[客户问题] 车辆是否支持无线充电？
[销售回答] 是的，中央控制台有一个无线充电板，支持Qi标准的设备都可以使用。

[客户问题] 车辆是否支持远程启动？
[销售回答] 是的，通过智能手机应用或遥控钥匙，您可以远程启动发动机并预设车内温度。



In [56]:
docs = topK_retriever.get_relevant_documents("车辆的保险费用大概是多少？")

In [57]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 车辆的保险费用大概是多少？
[销售回答] 保险费用因人而异，取决于驾驶记录、年龄和所选保险公司的政策，建议您联系保险公司获取报价。

[客户问题] 车辆的保修期是多久？
[销售回答] 我们的标准保修期为3年或10万公里，以先到者为准。

[客户问题] 这车的维修成本高吗？
[销售回答] 我们的车辆设计考虑了低维护成本，定期保养和更换常规部件的费用在合理范围内。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [58]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [59]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 车辆支持Apple CarPlay和Android Auto吗？
[销售回答] 是的，信息娱乐系统支持这两种手机连接方式，方便您在驾驶时使用手机功能。

[客户问题] 车辆是否支持无线充电？
[销售回答] 是的，中央控制台有一个无线充电板，支持Qi标准的设备都可以使用。



### 提取向量数据库中的`销售回答`

In [60]:
docs = retriever.get_relevant_documents(query)

In [61]:
docs[0].page_content

'[客户问题] 车辆支持Apple CarPlay和Android Auto吗？\n[销售回答] 是的，信息娱乐系统支持这两种手机连接方式，方便您在驾驶时使用手机功能。'

In [62]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 车辆支持Apple CarPlay和Android Auto吗？\n',
 '是的，信息娱乐系统支持这两种手机连接方式，方便您在驾驶时使用手机功能。']

In [63]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [64]:
ans

'是的，信息娱乐系统支持这两种手机连接方式，方便您在驾驶时使用手机功能。'

#### 尝试各种问题

In [65]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [66]:
query = "我想要知道这辆车的360影像清晰度如何？"

print(sales(query))

['倒车摄像头提供高清画面，确保您在狭小空间内也能安全倒车。']


In [67]:
print(sales(query, 0.75))

['倒车摄像头提供高清画面，确保您在狭小空间内也能安全倒车。', '该车配备了一套高品质音响系统，由多个扬声器组成，提供清晰且环绕的音效体验。']


In [68]:
query = "车辆支不支持座椅通风和座椅加热功能？"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: ['前排座椅配备三级加热功能，后座也可选装加热功能，确保冬季驾驶舒适。']

score:0.75 ans: ['前排座椅配备三级加热功能，后座也可选装加热功能，确保冬季驾驶舒适。', '车辆装备有远程预热系统，即使在极低温度下也能迅速提高车厢温度，确保温暖舒适的驾驶体验。', '我们的车辆装备了双区自动气候控制系统，前后排乘客可以根据个人喜好调节温度。', '是的，通过智能手机应用或遥控钥匙，您可以远程启动发动机并预设车内温度。']

score:0.5 ans: ['前排座椅配备三级加热功能，后座也可选装加热功能，确保冬季驾驶舒适。', '车辆装备有远程预热系统，即使在极低温度下也能迅速提高车厢温度，确保温暖舒适的驾驶体验。', '我们的车辆装备了双区自动气候控制系统，前后排乘客可以根据个人喜好调节温度。', '是的，通过智能手机应用或遥控钥匙，您可以远程启动发动机并预设车内温度。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [69]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5,base_url="https://api.xiaoai.plus/v1", api_key="sk-9o1als9Y9It7G1YoF38936F9061b4fE9B7C082Ec4cA83215")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [70]:
qa_chain({"query": "你们销售的汽车有没有能够像变形金刚一样，支持变形功能的？"})

d:\env\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们销售的汽车有没有能够像变形金刚一样，支持变形功能的？',
 'result': '很抱歉，我们销售的汽车目前没有能够像变形金刚一样支持变形功能的车型。'}

In [71]:
qa_chain({"query": "车子保养贵不贵？工时费多少？"})

{'query': '车子保养贵不贵？工时费多少？',
 'result': '我们的车辆保养费用在合理范围内，具体费用会根据车型和所在地区的不同而有所差异。工时费也会根据不同的维修项目而有所变化。您可以咨询当地的经销商或服务中心，他们可以给您提供更准确的费用信息。'}

In [72]:
print(sales("车子保养贵不贵？工时费多少？"))

['我们的车辆设计考虑了低维护成本，定期保养和更换常规部件的费用在合理范围内。']


## 加载 FAISS 向量数据库已有结果

In [73]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("car_sales_qa", OpenAIEmbeddings(),allow_dangerous_deserialization = True)

In [74]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [75]:
qa_chain({"query": "我想买劳斯莱斯幻影，你们有吗？"})

d:\env\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买劳斯莱斯幻影，你们有吗？',
 'result': '很抱歉，我无法提供您所需的产品。我是一个语言模型AI助手，无法提供实际商品。建议您联系当地的汽车经销商或者访问劳斯莱斯官方网站以获取更多信息。'}

In [76]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [77]:
qa_chain({"query": "我想买劳斯莱斯幻影，你们有吗？"})

d:\env\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买劳斯莱斯幻影，你们有吗？',
 'result': '很抱歉，我是一个AI助手，无法提供实物商品。建议您联系当地的汽车经销商或者在线汽车交易平台，以获取更多关于劳斯莱斯幻影的信息和购买渠道。'}

In [78]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [79]:
result = qa_chain({"query": "想买劳斯莱斯幻影，你们有吗？"})

d:\env\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [80]:
result

{'query': '想买劳斯莱斯幻影，你们有吗？',
 'result': '很抱歉，我是一个语言模型AI，无法提供实际产品。建议您联系当地的劳斯莱斯经销商或者在相关的汽车销售平台上进行查询。',
 'source_documents': []}